In [1]:
%load_ext autoreload
%autoreload 2

from SPARQLWrapper import SPARQLWrapper, JSON
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import  parser.matcher as matcher
import blazegraph_querier.query_to_json as query_to_json

In [2]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [44]:
queryString = "prefix dcterm: <http://purl.org/dc/terms/> select (COUNT(*) as ?count) {?s dcterm:type <http://psi.rechtspraak.nl/uitspraak>}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
ret = sparql.query()
ret.convert()

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '29605'}}]}}

In [20]:
searchstring = 'insolventierecht'
pred = 'dcterm:subject'
obj = '<http://psi.rechtspraak.nl/rechtsgebied#civielRecht_insolventierecht>'
nodes_json, links_json = query_to_json.query(None, only_linked=False, sparql=None, pred=pred, obj=obj)

In [27]:
searchstring = 'BWB7-685'
pred = 'dcterm:reference'
obj = '<bwb-dl1.0:v:BWB:BWBR0005290&artikel=685>'
nodes_json, links_json = query_to_json.query(None, only_linked=False, sparql=None, pred=pred, obj=obj)

In [30]:
searchstring =  'werkgever aansprakelijkheid'

nodes_json, links_json = query_to_json.query(searchstring, only_linked=False, sparql=None)

In [31]:
len(nodes_json)

138

In [33]:
nodes_json[0]

{'abstract': 'Arbeidsrecht. Verjaring van op grond van art. 7:658 BW tegen werkgever ingestelde vordering tot vergoeding schade als gevolg van RSI-klachten. Om de verjaringstermijn te doen aanvangen is geen bekendheid van benadeelde met alle schadecomponenten vereist. Voldoende is dat benadeelde bekend is met schade die hij lijdt of heeft geleden als gevolg van foutief handelen. Verjaring die op de voet van art. 3:310 BW begint te lopen, geldt mede voor de vordering tot vergoeding van schade waarvan de benadeelde kon verwachten dat hij die als gevolg van datzelfde tekortschietend handelen zou kunnen gaan lijden. Maatstaf verjaring: HR 9 oktober 2009, LJN BJ 4850.',
 'articles': ['3:310 bw', '7:658 bw'],
 'authorities': 0.0,
 'betweenness_centrality': 0.0,
 'count_annotation': 1,
 'count_version': 9,
 'creator': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden',
 'date': '2010-09-10',
 'degree': 0,
 'degree_centrality': 0.0,
 'ecli': 'ECLI:NL:HR:2010:BM7041',
 'hubs'

In [35]:
import urllib

In [36]:
filename = '/media/sf_VBox_Shared/CaseLaw/graphs/json_sigma/sigma_'+ urllib.parse.quote(searchstring) + '.json'
query_to_json.to_sigma_json(nodes_json, links_json, filename)

In [43]:
filename_csv = '/media/sf_VBox_Shared/CaseLaw/spot/'+ urllib.parse.quote(searchstring) + '.csv'
query_to_json.to_csv(nodes_json, filename_csv)